lr x100, latent 32->64

In [1]:
import numpy as np
data=np.load('/home/share/TmpData/Qinglin/ABIDE/X.npy')

In [2]:
'''This script demonstrates how to build a variational autoencoder with Keras.

 #Reference

 - Auto-Encoding Variational Bayes
   https://arxiv.org/abs/1312.6114
'''
from __future__ import print_function

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from keras import regularizers

from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import metrics
from keras.datasets import mnist

from keras import backend as K
K.clear_session()

batch_size = 20
original_dim=data.shape[1]
latent_dim = 64

epochs = 100
epsilon_std = 1.0


x = Input(shape=(original_dim,))
h = Dense(256, activation='tanh',
                activity_regularizer=regularizers.l1(1*10e-5))(x)
h = Dense(128, activation='tanh',
                activity_regularizer=regularizers.l1(1*10e-5))(h)
h = Dense(64, activation='tanh',
                activity_regularizer=regularizers.l1(1*10e-5))(h)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)


def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later

h_decoded = Dense(64, activation='tanh')(z)
h_decoded = Dense(128, activation='tanh')(h_decoded)
h_decoded = Dense(256, activation='tanh')(h_decoded)
x_decoded_mean = Dense(original_dim, activation='tanh')(h_decoded)

# instantiate VAE model
vae = Model(x, x_decoded_mean)

# Compute VAE loss
xent_loss = original_dim * metrics.binary_crossentropy(x,x_decoded_mean )
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(xent_loss + kl_loss)

vae.add_loss(vae_loss)
from keras import optimizers
rmsprop=optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
vae.compile(optimizer=rmsprop)
vae.summary()


vae.fit(data,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size)

encoder = Model(x, z)



Using TensorFlow backend.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28546)        0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 256)          7308032     input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 128)          32896       dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 64)           8256        dense_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (D

Epoch 59/100
17200/17200 [==============================] - 28s 2ms/step - loss: -185537.3914
Epoch 60/100
17200/17200 [==============================] - 26s 2ms/step - loss: -185018.5341
Epoch 61/100
17200/17200 [==============================] - 26s 2ms/step - loss: -185387.1387
Epoch 62/100
17200/17200 [==============================] - 26s 1ms/step - loss: -185915.3527
Epoch 63/100
17200/17200 [==============================] - 26s 1ms/step - loss: -186074.9211
Epoch 64/100
17200/17200 [==============================] - 27s 2ms/step - loss: -185670.6623
Epoch 65/100
17200/17200 [==============================] - 28s 2ms/step - loss: -186046.1352
Epoch 66/100
17200/17200 [==============================] - 28s 2ms/step - loss: -186226.5613
Epoch 67/100
17200/17200 [==============================] - 28s 2ms/step - loss: -186061.0110
Epoch 68/100
17200/17200 [==============================] - 28s 2ms/step - loss: -186160.6009
Epoch 69/100
17200/17200 [==============================] - 

In [3]:
encoder.save('/home/share/TmpData/Qinglin/ABIDE/ABIDEencoder_64.h5')

In [4]:
encoder=Model(x, z)
y=encoder.predict(data,
        batch_size=batch_size)
y.shape

(17200, 64)

In [5]:
from nilearn.datasets import fetch_abide_pcp

# We specify the site and number of subjects we want to download
abide = fetch_abide_pcp(derivatives=['func_preproc'], data_dir='/home/share/TmpData/Qinglin/nilearn_data/')

# We look at the available data in this dataset
print(abide.keys())


func_filenames = abide.func_preproc  # list of 4D nifti files for each subject
Y = abide.phenotypic['DX_GROUP']
for index, item in enumerate(Y):
    if item>1:
        Y[index] = 0
        
# print basic information on the dataset
print('First functional nifti image (4D) is at: %s' %
      func_filenames[0])  # 4D data

print(Y)

from nilearn._utils.niimg_conversions import _resolve_globbing
imgs = _resolve_globbing(func_filenames)

mask_img ='/home/share/TmpData/Qinglin/ADHD200_Athena_preproc_flirtfix/ADHD200_mask_152_4mm.nii.gz'

from nilearn.input_data import NiftiMasker
masker = NiftiMasker(mask_img=mask_img, 
                     standardize=True,
                     detrend=1,
                     smoothing_fwhm=6.,
                     memory="/storage/nilearn_cache", 
                     memory_level=2)
fmri_masked = masker.fit()

/usr/local/lib/python3.5/dist-packages/numpy/lib/npyio.py:2278: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


dict_keys(['func_preproc', 'phenotypic', 'description'])
First functional nifti image (4D) is at: /home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050003_func_preproc.nii.gz
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 

/usr/local/lib/python3.5/dist-packages/nilearn/_utils/cache_mixin.py:84: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/storage/nilearn_cache'", use "location='/storage/nilearn_cache'" instead.
  memory = Memory(cachedir=cache_dir, verbose=verbose)
/usr/local/lib/python3.5/dist-packages/nilearn/_utils/cache_mixin.py:232: DeprecationWarning: The 'cachedir' attribute has been deprecated in version 0.12 and will be removed in version 0.14.
Use os.path.join(memory.location, 'joblib') attribute instead.
  if (memory.cachedir is None and memory_level is not None


In [6]:
from nilearn.decomposition.base import _mask_and_reduce_single
all_time_series=[]

for img in imgs:
    print(img)
    data = _mask_and_reduce_single(
        masker, img, confound=None,
        reduction_ratio=1,
        random_state=0,
        memory_level=3)
    time_series=encoder.predict(data,
        batch_size=batch_size)
    all_time_series.append(time_series)


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050003_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050004_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050005_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050006_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050007_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050008_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050010_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050011_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Pitt_0050012_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050143_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050144_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050145_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050146_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050147_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050148_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050149_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050150_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/OHSU_0050152_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Trinity_0050266_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Trinity_0050267_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Trinity_0050268_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Trinity_0050269_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Trinity_0050270_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/Trinity_0050271_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050272_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050273_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050274_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_d

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050366_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050367_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050368_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050369_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050370_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050372_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050373_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050374_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UM_1_0050375_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac

...

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051017_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051018_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051019_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051020_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051021_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051023_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051024_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051025_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051026_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_n

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051102_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051103_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051104_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051105_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051106_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051107_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051109_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051110_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/NYU_0051111_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_n

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051207_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051208_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051210_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051211_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051212_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051214_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051215_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051216_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_1_0051217_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_d

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_2_0051313_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/UCLA_2_0051315_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_a_0051318_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_a_0051319_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_a_0051320_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_a_0051321_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_b_0051322_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_b_0051323_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/MaxMun_b_0051325_func_preproc.nii.gz
/home/share/TmpData/Qin

/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051574_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051576_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051577_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051578_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051579_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051580_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051582_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051583_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_noglobal/SBL_0051584_func_preproc.nii.gz
/home/share/TmpData/Qinglin/nilearn_data/ABIDE_pcp/cpac/nofilt_n

In [7]:
np.save('/home/share/TmpData/Qinglin/ABIDE/all_time_series_64.npy',all_time_series)

In [8]:
np.save('/home/share/TmpData/Qinglin/ABIDE/Y.npy',Y)